In [1]:
import numpy as np
from gensim.models import KeyedVectors
from sklearn.cross_decomposition import CCA

C:\Users\lappy\Anaconda21\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
vecfile = 'GoogleNews-vectors-negative300.bin'
vecs = KeyedVectors.load_word2vec_format(vecfile, binary =True)
vecs.init_sims(replace=True)

C:\Users\lappy\Anaconda21\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mailt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mailt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import inflect
import re
from nltk.corpus import stopwords
from scipy.stats import pearsonr

In [5]:
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
           new_words.append(new_word)
    return new_words
                    
def replace_numbers(words):
    p = inflect.engine()
    new_words =[]
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words        
                                                    
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words  
                          

In [6]:
def normalize(words):
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [7]:
def compute_cosine_similarity(vector1, vector2):
    
    
    cos=vector1.dot(vector2)/(np.linalg.norm(vector1, ord=2) * np.linalg.norm(vector2, ord=2))
    
    if np.isnan(cos):
        return 0.500 # arbitrarily low similarity
    else:
        return cos
  

In [18]:
def cca_semantic(inpu, vecs, output):
    
    
    write_str  = []
    sims = []
    s_min = 1
    s_max = 0
    sim = 0
    original_value=[]
    
    
    for line in inpu:
        line_components = line.split("\t")
        sent1 = line_components[1]
        sent2 = line_components[2]
        original_value.append(line_components[0])
        
        
        words_in_sent1 = normalize(sent1.split())
        words_in_sent2 = normalize(sent2.split())
        
        stems1 = []
        for word in words_in_sent1:
            if word in vecs:
                stems1.append(word)
                
        stems2 = []
        for word in words_in_sent2:
            if word in vecs:
                stems2.append(word)
        
        
        
        len_1 = len(stems1)
        len_2 = len(stems2)
        len_min = min(len_1, len_2, 5)  # yaha badlo  3 se 
         
        
        
        
        v1 = np.asarray(vecs["hi"])
        v2 = np.asarray(vecs["hi"])
        
        for word in stems1:
            x = np.asarray(vecs[word]) 
            v1 = np.vstack((v1, x))
        for word in stems2:
            x = np.asarray(vecs[word]) 
            v2 = np.vstack((v2, x))
            
        v1 = np.delete(v1, 0, 0)
        v2 = np.delete(v2, 0, 0)
        
        b = len_min    
            
        cca = CCA(n_components =b)
            
        cca.fit(v1.T, v2.T)
        X_c, Y_c = cca.transform(v1.T, v2.T)
        
        sim =0
        write_str1 = []
        write_str2 = []
        s = ""
        t = ""
        
        for i in range(b):
            
            
            
            #vector 1
            v1 = []
             
            for j in X_c:
                v1.append(j[i])
                
            w11 = np.asarray(v1)
        
            con11 = vecs.most_similar(positive = [w11], topn =1)
            context11 = con11[0][0]
        
        
            #vector 2
            v2 = []
             
            for j in Y_c:
                v2.append(j[i])
                
            w21 = np.asarray(v2)
        
            con21 = vecs.most_similar(positive = [w21], topn =1)
            context21 = con21[0][0]
            
            #cosine similarity
            sim_1 = compute_cosine_similarity(w11, w21)
            sim =sim + sim_1
            
            write_str1.append(context11)
            write_str2.append(context21)
            
        sim = sim/b
        sims.append(sim)

        s_max = max(s_max, sim)
        s_min = min(s_min, sim)
        
        
        for i in write_str1:
            s += i 
            s += ", "
        
        for i in write_str2:
            t += i 
            t += ", "
        
        write_str.append(sent1 + "\t" + '(' + s + ')' + "\t" + sent2 +  "\t" +  '(' + t + ')' + "\t")        
    
    
    
    sims_scaled = [5*(i - s_min)/(s_max - s_min) for i in sims]
    
    for i in range(0,len(write_str)):
        output.write(write_str[i] + str(sims_scaled[i]) + "\n")
        sims_scaled[i]   =  float(sims_scaled[i])
        original_value[i] = float(original_value[i])
        
    covariance = pearsonr(sims_scaled,original_value)
    print(covariance)    
        

In [19]:

with open("answers-forums.test.txt", 'r') as inputfile:
    inpu = inputfile.readlines()

output_cca = open("pred_answers-forums.txt", 'w')

cca_semantic(inpu, vecs, output_cca)

output_cca.close()

    
    

(0.737283733383587, 3.940506467496995e-44)


In [11]:
x = np.asarray(vecs["red"]) + np.asarray(vecs["white"]) + np.asarray(vecs["pink"])

In [13]:
 y = vecs.most_similar(positive =[x], topn=1)

In [14]:
y

[('pink', 0.8604159355163574)]

In [15]:
type(y)

list

In [16]:
y[0]

('pink', 0.8604159355163574)

In [19]:
y[0][0]

'pink'

In [46]:
a = np.array([[1,2,3, 5, 6, 7], [4,5,6, 8, 8, 9], [7,8,9, 10, 11 ,12]])
b = np.array([[10,11,12, 14, 7 ,8], [14,15,16, 9, 1, 2]])

cca =CCA(n_components=2)

cca.fit(a.T, b.T)

X_c, Y_c = cca.transform(a.T, b.T)

In [47]:
X_c

array([[ 0.01459147,  0.22569892],
       [-0.02918293,  0.19409217],
       [-0.07295734,  0.16248541],
       [-0.16050614, -0.30137706],
       [ 0.14591467, -0.12464635],
       [ 0.10214027, -0.1562531 ]])

In [53]:
a=[]
n =2
for j in range(2):
   for i in X_c:
       a.append(i[j])
   print(a)
   a = []
   

[0.014591467303593737, -0.029182934607190434, -0.0729573365179748, -0.1605061403395427, 0.14591467303594954, 0.10214027112516533]
[0.22569891775642076, 0.19409216596015597, 0.16248541416389128, -0.3013770558601536, -0.12464634511202465, -0.15625309690828953]


In [16]:
Y_c

array([[ 0.12909944,  0.76173883],
       [-0.25819889,  0.65506535],
       [-0.64549722,  0.54839186],
       [-1.42009389, -1.01715422],
       [ 1.29099445, -0.42068417],
       [ 0.90369611, -0.52735765]])

In [20]:
Y_c

array([[ 0.12909944],
       [-0.25819889],
       [-0.64549722],
       [-1.42009389],
       [ 1.29099445],
       [ 0.90369611]])

In [8]:
x = np.asarray(vecs["red"]) + np.asarray(vecs["white"]) + np.asarray(vecs["pink"])

In [10]:
a =vecs.most_similar(positive =[x], topn=1)

In [11]:
type(a)

list

In [12]:
a[0]

('red', 0.8569148182868958)

In [13]:
a[0][0]

'red'